## Main Pipeline Function

The main function that connects all components together.

In [ ]:
def qa_to_images_pipeline(source_type="file", file_path="Hampi_Architecture_QA.txt", 
                         vector_store_path="./chroma_architecture_qa_db", query=None,
                         num_images=1, output_dir="hampi_images", monument_filters=None,
                         reference_dir=DEFAULT_REFERENCE_IMAGES_DIR, reference_count=5,
                         device="cuda"):
    """
    Complete pipeline to convert architectural Q&A data to realistic images
    using reference-based image generation
    
    Args:
        source_type: 'file' or 'vector_store'
        file_path: Path to the QA text file (if source_type is 'file')
        vector_store_path: Path to the vector store (if source_type is 'vector_store')
        query: Search query for the vector store (if source_type is 'vector_store')
        num_images: Number of images to generate per Q&A pair
        output_dir: Directory to save the generated images
        monument_filters: List of specific monuments to include (e.g., ['Vitthala Temple'])
        reference_dir: Directory containing reference images
        reference_count: Number of reference images to use
        device: Device to use for generation
    """
    print("Loading models...")
    models = load_models(device=device)
    
    print("Retrieving Q&A pairs...")
    qa_pairs = get_qa_pairs(source_type, file_path, vector_store_path, query)
    
    if not qa_pairs:
        print("No Q&A pairs found!")
        return []
    
    print(f"Found {len(qa_pairs)} Q&A pairs.")
    
    # Filter by monument if specified
    if monument_filters:
        filtered_pairs = []
        for pair in qa_pairs:
            if any(monument.lower() in pair["question"].lower() for monument in monument_filters):
                filtered_pairs.append(pair)
        qa_pairs = filtered_pairs
        print(f"Filtered to {len(qa_pairs)} Q&A pairs related to specified monuments.")
    
    generated_image_paths = []
    
    for i, pair in enumerate(qa_pairs):
        print(f"\nProcessing Q&A pair {i+1}/{len(qa_pairs)}")
        print(f"Question: {pair['question']}")
        
        # Extract monument name from question if possible
        monument_match = re.search(r'(Vitthala|Virupaksha|Krishna|Hazara Rama|Lotus Mahal|Elephant Stables)', pair['question'])
        monument_name = monument_match.group(1) if monument_match else None
        
        # Extract architectural details
        full_qa_text = f"Question: {pair['question']}\nAnswer: {pair['answer']}"
        architectural_details = extract_architectural_details(full_qa_text)
        
        if not architectural_details:
            print("Could not extract architectural details, skipping...")
            continue
        
        print(f"Extracted details: {architectural_details[:100]}...")
        
        # Format image generation prompt
        prompt, negative_prompt = format_image_prompt(architectural_details, monument_name)
        print(f"Generated prompt: {prompt[:100]}...")
        
        # Generate images using reference-based approach
        image_paths = generate_images(
            prompt=prompt, 
            negative_prompt=negative_prompt, 
            models=models, 
            num_images=num_images, 
            output_dir=output_dir,
            reference_dir=reference_dir,
            reference_count=reference_count,
            monument_name=monument_name
        )
        
        generated_image_paths.extend(image_paths)
    
    print(f"\nGeneration complete. Created {len(generated_image_paths)} images in {output_dir}.")
    return generated_image_paths

## Example Usage

Below are examples of how to use the pipeline.

In [ ]:
# Example 1: Generate images from text file
# generated_images = qa_to_images_pipeline(
#     source_type="file",
#     file_path="Hampi_Architecture_QA.txt",
#     num_images=2,
#     monument_filters=["Vitthala Temple", "pillars"],
#     output_dir="hampi_output",
#     reference_dir=DEFAULT_REFERENCE_IMAGES_DIR,
#     reference_count=5
# )

In [ ]:
# Example 2: Generate images from vector store with query
# generated_images = qa_to_images_pipeline(
#     source_type="vector_store",
#     vector_store_path="./chroma_architecture_qa_db",
#     query="What are the distinctive features of pillars in Hampi temples?",
#     num_images=2,
#     output_dir="hampi_output",
#     reference_dir=DEFAULT_REFERENCE_IMAGES_DIR,
#     reference_count=5
# )

## Display Generated Images

Use this cell to display generated images.

In [ ]:
# Display images
from IPython.display import Image, display

def display_generated_images(image_paths, max_display=3):
    """Display generated images in the notebook"""
    if not image_paths:
        print("No images to display.")
        return
    
    print(f"Displaying {min(len(image_paths), max_display)} of {len(image_paths)} generated images:")
    
    for i, path in enumerate(image_paths[:max_display]):
        if os.path.exists(path):
            print(f"\nImage {i+1}: {os.path.basename(path)}")
            display(Image(filename=path))
        else:
            print(f"Image not found: {path}")
            
    if len(image_paths) > max_display:
        print(f"\n... and {len(image_paths) - max_display} more images")

# Uncomment to display images from a previous run
# image_dir = "hampi_output"  # Directory containing generated images
# image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) 
#                if f.endswith('.png') and 'comparison' not in f]
# display_generated_images(image_files)

## Direct Image Generation

You can also directly generate images from specific prompts without Q&A data.

In [ ]:
def generate_direct_images(prompt_text, num_images=2, output_dir=OUTPUT_DIR, 
                          reference_dir=DEFAULT_REFERENCE_IMAGES_DIR, reference_count=5,
                          device="cuda"):
    """Generate images directly from a prompt without Q&A extraction"""
    print("Loading models...")
    models = load_models(device=device)
    
    # Create a simple prompt based on the text
    prompt, negative_prompt = format_image_prompt(prompt_text)
    print(f"Generated prompt: {prompt}")
    
    # Generate images
    image_paths = generate_images(
        prompt=prompt,
        negative_prompt=negative_prompt,
        models=models,
        num_images=num_images,
        output_dir=output_dir,
        reference_dir=reference_dir,
        reference_count=reference_count
    )
    
    return image_paths

# Example: Direct generation
# direct_images = generate_direct_images(
#     prompt_text="Vittala temple with its famous stone chariot and musical pillars",
#     num_images=2
# )
# display_generated_images(direct_images)

## Quick Generate with Reference Image

This utility lets you quickly generate images from a prompt using a specific reference image.

In [ ]:
def quick_generate_with_reference(prompt_text, reference_image_path, output_dir=OUTPUT_DIR, 
                                 strength=0.75, guidance_scale=7.5, device="cuda"):
    """Quickly generate an image based on a specific reference image"""
    print("Loading models...")
    models = load_models(device=device)
    
    # Create prompt
    prompt, negative_prompt = format_image_prompt(prompt_text)
    
    # Load and preprocess reference image
    ref_image = Image.open(reference_image_path).convert("RGB")
    processed_ref = preprocess_reference_image(ref_image)
    
    # Generate seed
    seed = random.randint(0, 2147483647)
    
    # Generate image
    try:
        print(f"Generating from reference image: {os.path.basename(reference_image_path)}")
        generated_img = generate_from_reference(
            models=models,
            prompt=prompt,
            reference_image=processed_ref,
            negative_prompt=negative_prompt,
            strength=strength,
            guidance_scale=guidance_scale,
            num_inference_steps=50,
            seed=seed
        )
        
        # Save image
        os.makedirs(output_dir, exist_ok=True)
        base_name = os.path.splitext(os.path.basename(reference_image_path))[0]
        output_path = os.path.join(output_dir, f"{base_name}_generated_seed{seed}.png")
        generated_img.save(output_path)
        print(f"Saved to: {output_path}")
        
        # Save comparison
        comparison = Image.new('RGB', (processed_ref.width * 2, processed_ref.height))
        comparison.paste(processed_ref, (0, 0))
        comparison.paste(generated_img, (processed_ref.width, 0))
        comparison_path = os.path.join(output_dir, f"{base_name}_comparison_seed{seed}.png")
        comparison.save(comparison_path)
        
        # Display images
        print("Reference image:")
        display(Image(filename=reference_image_path))
        print("Generated image:")
        display(Image(filename=output_path))
        print("Side-by-side comparison:")
        display(Image(filename=comparison_path))
        
        return output_path, comparison_path
    except Exception as e:
        print(f"Error generating image: {e}")
        return None, None

# Example: Quick generation with a specific reference image
# quick_generate_with_reference(
#     prompt_text="The detailed ornate pillars of Hampi temples with intricate carvings",
#     reference_image_path="D:\\college\\imp-doc\\sem6\\GENAI\\project\\new\\all_images\\example.jpg",
#     strength=0.7  # 0.0 = keep reference exactly, 1.0 = completely replace
# )